In [1]:
from pydantic import BaseModel, Field, create_model

In [4]:


class Address(BaseModel):
    street: str
    
class User(BaseModel):
    id: int
    name: str = Field(..., min_length=3, max_length=15)
    address: Address

In [6]:
User.model_json_schema()

{'$defs': {'Address': {'properties': {'street': {'title': 'Street',
     'type': 'string'}},
   'required': ['street'],
   'title': 'Address',
   'type': 'object'}},
 'properties': {'id': {'title': 'Id', 'type': 'integer'},
  'name': {'maxLength': 15, 'minLength': 3, 'title': 'Name', 'type': 'string'},
  'address': {'$ref': '#/$defs/Address'}},
 'required': ['id', 'name', 'address'],
 'title': 'User',
 'type': 'object'}

In [10]:
from typing import Literal

class Route(BaseModel):
    method: Literal['reply', 'tool'] = Field(..., description="The method to use for the route")

In [31]:
from typing import Union

def make_route(method: str = 'reply') -> type:
    """Creates a Pydantic model for Route with a specified method."""
    return create_model('Route', method=(Literal[method], ...))
reply = make_route('reply')
tools = make_route('tool')
Union[reply, tools]

typing.Union[__main__.Route, __main__.Route]

In [1]:
class Tools(BaseModel):
    tools: dict[str, dict[str, str]] = Field(..., description="The tools to use for the route")

class Reply(BaseModel):
    reply: str = Field(..., description="The reply to the route")

class ToolCalling(BaseModel):
    tools: Tools


NameError: name 'BaseModel' is not defined

In [2]:
Route.model_json_schema()

NameError: name 'Route' is not defined

In [37]:
from pydantic import BaseModel, Field

def make_route(method: str = 'reply') -> type:
    return create_model(
        'Route',
        method=(Literal[method], ...),
        payload=(str, 'default_payload')
    )

ReplyRoute = make_route('reply')
ToolsRoute = make_route('tools')

class Message(BaseModel):
    route: Union[ReplyRoute, ToolsRoute] = Field(..., discriminator='method')

# Test with data
reply_data = {'route': {'method': 'reply', 'payload': 'Hello, reply!'}}
message1 = Message.model_validate(reply_data)
print(message1)

tools_data = {'route': {'method': 'tools', 'payload': 'Hello, tools!'}}
message2 = Message.model_validate(tools_data)
print(message2)

route=Route(method='reply', payload='Hello, reply!')
route=Route(method='tools', payload='Hello, tools!')


In [36]:
Message.model_json_schema()

{'$defs': {'__main____Route__1': {'properties': {'method': {'const': 'reply',
     'title': 'Method',
     'type': 'string'},
    'payload': {'default': 'default_payload',
     'title': 'Payload',
     'type': 'string'}},
   'required': ['method'],
   'title': 'Route',
   'type': 'object'},
  '__main____Route__2': {'properties': {'method': {'const': 'tools',
     'title': 'Method',
     'type': 'string'},
    'payload': {'default': 'default_payload',
     'title': 'Payload',
     'type': 'string'}},
   'required': ['method'],
   'title': 'Route',
   'type': 'object'}},
 'properties': {'route': {'anyOf': [{'$ref': '#/$defs/__main____Route__1'},
    {'$ref': '#/$defs/__main____Route__2'}],
   'title': 'Route'}},
 'required': ['route'],
 'title': 'Message',
 'type': 'object'}

In [38]:
Message.model_json_schema()

{'$defs': {'__main____Route__1': {'properties': {'method': {'const': 'reply',
     'title': 'Method',
     'type': 'string'},
    'payload': {'default': 'default_payload',
     'title': 'Payload',
     'type': 'string'}},
   'required': ['method'],
   'title': 'Route',
   'type': 'object'},
  '__main____Route__2': {'properties': {'method': {'const': 'tools',
     'title': 'Method',
     'type': 'string'},
    'payload': {'default': 'default_payload',
     'title': 'Payload',
     'type': 'string'}},
   'required': ['method'],
   'title': 'Route',
   'type': 'object'}},
 'properties': {'route': {'discriminator': {'mapping': {'reply': '#/$defs/__main____Route__1',
     'tools': '#/$defs/__main____Route__2'},
    'propertyName': 'method'},
   'oneOf': [{'$ref': '#/$defs/__main____Route__1'},
    {'$ref': '#/$defs/__main____Route__2'}],
   'title': 'Route'}},
 'required': ['route'],
 'title': 'Message',
 'type': 'object'}

In [20]:
ReplyRoute = create_model('ReplyRoute', __base__=(Reply, Route))
ToolCallingRoute = create_model('ToolCallingRoute', __base__=(ToolCalling, Route))

In [21]:
ReplyRoute.model_json_schema()
ToolCallingRoute.model_json_schema()


{'$defs': {'Tools': {'properties': {'tools': {'additionalProperties': {'additionalProperties': {'type': 'string'},
      'type': 'object'},
     'description': 'The tools to use for the route',
     'title': 'Tools',
     'type': 'object'}},
   'required': ['tools'],
   'title': 'Tools',
   'type': 'object'}},
 'properties': {'method': {'description': 'The method to use for the route',
   'enum': ['reply', 'tool'],
   'title': 'Method',
   'type': 'string'},
  'tools': {'$ref': '#/$defs/Tools'}},
 'required': ['method', 'tools'],
 'title': 'ToolCallingRoute',
 'type': 'object'}

In [ ]:
{
    method: 'tools',
}



In [ ]:
"""
routing_map takes either ports or a payload_type key for connecting and/or inferring
the payload type

"""

StructuredRouterTransformer(
    routing_map={
        'reply': {
            'schema': {'response': str}
        },
        'tools': {
            'schema_input_ports': [mcp_el.ports.tool_list_output],
            'transform': lambda structured_input: ToolCallPayload(tools=structured_input),
            'payload_type': ToolCallPayload,
            'ports': [mcp_el.ports.tool_call_input]
        }
    }
)

In [ ]:
{
    'method': 'tools' # the routing specifier
    'tools': ['weather_mcp_get_weather': {'location': 'New York'}]
}

{
    'method': 'reply' # the routing specifier
    'reply': 'Hello, reply!'
}

In [3]:
from pydantic import BaseModel, Field
from typing import Union, List, Dict

class MethodModel(BaseModel):
    method: str = Field(..., description="The method to use for the route")

class ToolsModel(BaseModel):
    tools: Dict[str, Dict[str, str]] = Field(..., description="The tools to use for the route")

class ReplyModel(BaseModel):
    reply: str = Field(..., description="The reply message")

# Define a union type for the combined models
# This allows for validation of either a ToolsModel or a ReplyModel alongside the MethodModel
class ToolCallingRouteWithTools(BaseModel):
    method: str
    tools: Dict[str, Dict[str, str]] = Field(..., description="The tools to use for the route")

class ToolCallingRouteWithReply(BaseModel):
    method: str
    reply: str = Field(..., description="The reply message")

# Define a union type for the combined models
Route = Union[ToolCallingRouteWithTools, ToolCallingRouteWithReply]



In [5]:
class ToolCallingRoute(BaseModel):
    """A model that validates either a ToolCallingRouteWithTools or ToolCallingRouteWithReply."""
    route: Union[ToolCallingRouteWithTools, ToolCallingRouteWithReply] = Field(..., description="The route specification")


In [6]:
ToolCallingRoute.model_json_schema()

{'$defs': {'ToolCallingRouteWithReply': {'properties': {'method': {'title': 'Method',
     'type': 'string'},
    'reply': {'description': 'The reply message',
     'title': 'Reply',
     'type': 'string'}},
   'required': ['method', 'reply'],
   'title': 'ToolCallingRouteWithReply',
   'type': 'object'},
  'ToolCallingRouteWithTools': {'properties': {'method': {'title': 'Method',
     'type': 'string'},
    'tools': {'additionalProperties': {'additionalProperties': {'type': 'string'},
      'type': 'object'},
     'description': 'The tools to use for the route',
     'title': 'Tools',
     'type': 'object'}},
   'required': ['method', 'tools'],
   'title': 'ToolCallingRouteWithTools',
   'type': 'object'}},
 'description': 'A model that validates either a ToolCallingRouteWithTools or ToolCallingRouteWithReply.',
 'properties': {'route': {'anyOf': [{'$ref': '#/$defs/ToolCallingRouteWithTools'},
    {'$ref': '#/$defs/ToolCallingRouteWithReply'}],
   'description': 'The route specificati

In [ ]:
# Example JSONs that follow the defined schema for ToolCallingRoute

example_json_reply = {
    "route": {
        "method": "reply",
        "reply": "Hello, this is a reply!"
    }
}

example_json_tools = {
    "route": {
        "method": "tools",
        "tools": {
            "tool1": {
                "name": "Tool One",
                "description": "This is the first tool."
            },
            "tool2": {
                "name": "Tool Two",
                "description": "This is the second tool."
            }
        }
    }
}

# These examples demonstrate how to structure the JSON according to the defined models.
# The first example is for a reply, while the second is for tools.


In [9]:
from pydantic import BaseModel, Field
from typing import Union

class ToolCallingRouteWithReply(BaseModel):
    """A model for routes that return a reply."""
    method: str = Field(..., description="The method to use for the route", enum=["reply"])
    reply: str = Field(..., description="The reply message")

class ToolCallingRouteWithTools(BaseModel):
    """A model for routes that utilize tools."""
    method: str = Field(..., description="The method to use for the route", enum=["tools"])
    tools: dict = Field(..., description="The tools to use for the route")

class ToolCallingRoute(BaseModel):
    """A model that validates either a ToolCallingRouteWithTools or ToolCallingRouteWithReply without an external 'route' key."""
    route: Union[ToolCallingRouteWithTools, ToolCallingRouteWithReply] = Field(..., description="The route specification")

# Note: The previous implementation incorrectly used a field called '__root__' in a root model.
# This has been corrected by directly including the 'route' field in the ToolCallingRoute model.







In [11]:
ToolCallingRoute.model_json_schema()

{'$defs': {'ToolCallingRouteWithReply': {'description': 'A model for routes that return a reply.',
   'properties': {'method': {'description': 'The method to use for the route',
     'enum': ['reply'],
     'title': 'Method',
     'type': 'string'},
    'reply': {'description': 'The reply message',
     'title': 'Reply',
     'type': 'string'}},
   'required': ['method', 'reply'],
   'title': 'ToolCallingRouteWithReply',
   'type': 'object'},
  'ToolCallingRouteWithTools': {'description': 'A model for routes that utilize tools.',
   'properties': {'method': {'description': 'The method to use for the route',
     'enum': ['tools'],
     'title': 'Method',
     'type': 'string'},
    'tools': {'description': 'The tools to use for the route',
     'title': 'Tools',
     'type': 'object'}},
   'required': ['method', 'tools'],
   'title': 'ToolCallingRouteWithTools',
   'type': 'object'}},
 'description': "A model that validates either a ToolCallingRouteWithTools or ToolCallingRouteWithRepl

In [ ]:
# Creating example JSONs that conform to the ToolCallingRoute schema

example_reply_route = {
    "route": {
        "method": "reply",
        "reply": "This is a reply message."
    }
}

example_tools_route = {
    "route": {
        "method": "tools",
        "tools": {
            "tool1": {
                "name": "Tool One",
                "description": "This is the first tool."
            },
            "tool2": {
                "name": "Tool Two",
                "description": "This is the second tool."
            }
        }
    }
}

# Example JSONs for testing
print(example_reply_route)
print(example_tools_route)


In [13]:
from typing import Dict, Union, Literal, Any
from pydantic import BaseModel, Field, RootModel

# Defines the structure for a single tool with its name and description.
class Tool(BaseModel):
    name: str
    description: str

# Model for a reply route.
class ReplyRoute(BaseModel):
    method: Literal["reply"]
    reply: str

# Model for a tools route where a mapping between tool keys and Tool definitions is provided.
class ToolsRoute(BaseModel):
    method: Literal["tools"]
    tools: Dict[str, Tool]

# The discriminated union that accepts either a ReplyRoute or a ToolsRoute.
# The use of __root__ with a discriminator on "method" means that this model directly
# validates JSON objects like {"method": "reply", ...} or {"method": "tools", ...}
# without requiring an outer key (e.g. "route").
class Route(RootModel):
    root: Union[ReplyRoute, ToolsRoute] = Field(..., discriminator="method")

# # Example usage:
# if __name__ == "__main__":
#     # JSON for a reply - note no outer "route" key is used.
#     reply_json: Any = {
#         "method": "reply",
#         "reply": "Hello, this is a reply!"
#     }
    
#     # JSON for tools.
#     tools_json: Any = {
#         "method": "tools",
#         "tools": {
#             "tool1": {"name": "Tool One", "description": "This is the first tool."},
#             "tool2": {"name": "Tool Two", "description": "This is the second tool."}
#         }
#     }
    
#     # Validate the reply JSON.
#     route_reply = Route.parse_obj(reply_json)
#     print("Validated reply route:", route_reply)
    
#     # Validate the tools JSON.
#     route_tools = Route.parse_obj(tools_json)
#     print("Validated tools route:", route_tools)
    
#     # Output the generated JSON schema for inspection.
#     print(Route.schema_json(indent=2))

In [14]:
Route.model_json_schema()

{'$defs': {'ReplyRoute': {'properties': {'method': {'const': 'reply',
     'title': 'Method',
     'type': 'string'},
    'reply': {'title': 'Reply', 'type': 'string'}},
   'required': ['method', 'reply'],
   'title': 'ReplyRoute',
   'type': 'object'},
  'Tool': {'properties': {'name': {'title': 'Name', 'type': 'string'},
    'description': {'title': 'Description', 'type': 'string'}},
   'required': ['name', 'description'],
   'title': 'Tool',
   'type': 'object'},
  'ToolsRoute': {'properties': {'method': {'const': 'tools',
     'title': 'Method',
     'type': 'string'},
    'tools': {'additionalProperties': {'$ref': '#/$defs/Tool'},
     'title': 'Tools',
     'type': 'object'}},
   'required': ['method', 'tools'],
   'title': 'ToolsRoute',
   'type': 'object'}},
 'discriminator': {'mapping': {'reply': '#/$defs/ReplyRoute',
   'tools': '#/$defs/ToolsRoute'},
  'propertyName': 'method'},
 'oneOf': [{'$ref': '#/$defs/ReplyRoute'}, {'$ref': '#/$defs/ToolsRoute'}],
 'title': 'Route'}

In [ ]:
# Example JSON objects that conform to the defined Route schema.

# JSON for a reply route
reply_json_example = {
    "method": "reply",
    "reply": "This is a sample reply message."
}

# JSON for a tools route
tools_json_example = {
    "method": "tools",
    "tools": {
        "tool1": {"name": "Sample Tool One", "description": "Description for tool one."},
        "tool2": {"name": "Sample Tool Two", "description": "Description for tool two."}
    }
}

# Print the examples to verify their structure
print("Reply JSON Example:", reply_json_example)
print("Tools JSON Example:", tools_json_example)


In [46]:
from pydantic import BaseModel, RootModel, Field
from typing import Literal, Union

class Reply(BaseModel):
    reply: str

class Tools(BaseModel):
    tools: dict

RouteReply = create_model(
    'RouteReplay',
    method=(Literal[Reply.__name__.lower()], ...),
    __base__=Reply
    )

RouteTools = create_model(
    'RouteTools',
    method=(Literal[Tools.__name__], ...),
    __base__=Tools
)
class Routes(RootModel):
    root: Union[RouteReply, RouteTools] = Field(discriminator='method')

# Check the schema
Routes.model_json_schema()

{'$defs': {'RouteReplay': {'properties': {'reply': {'title': 'Reply',
     'type': 'string'},
    'method': {'const': 'reply', 'title': 'Method', 'type': 'string'}},
   'required': ['reply', 'method'],
   'title': 'RouteReplay',
   'type': 'object'},
  'RouteTools': {'properties': {'tools': {'title': 'Tools', 'type': 'object'},
    'method': {'const': 'Tools', 'title': 'Method', 'type': 'string'}},
   'required': ['tools', 'method'],
   'title': 'RouteTools',
   'type': 'object'}},
 'discriminator': {'mapping': {'Tools': '#/$defs/RouteTools',
   'reply': '#/$defs/RouteReplay'},
  'propertyName': 'method'},
 'oneOf': [{'$ref': '#/$defs/RouteReplay'}, {'$ref': '#/$defs/RouteTools'}],
 'title': 'Routes'}

In [43]:
import json
validate_dict = json.loads("""
  {
    "method": "Reply",
    "reply": "Hi"
  }
""")

In [44]:
Routes(root=validate_dict)

Routes(root=RouteReplay(reply='Hi', method='Reply'))

In [34]:
Reply.__name__

'Reply'

In [17]:
_.model_json_schema()

{'properties': {'method': {'title': 'Method', 'type': 'string'},
  'reply': {'title': 'Reply', 'type': 'string'}},
 'required': ['method', 'reply'],
 'title': 'Route',
 'type': 'object'}